In [1]:
# from wandbhelper.util import init_wandb, log_all_plots, log_plot_as_image

# wandb_run_id = init_wandb(run_name="price_match_simulation")
# print(f"Run ID {wandb_run_id}")

In [2]:
import pandas as pd
import pickle
import time

from ml_simulation.dataset_split import customer_split
from ml_simulation.util import HiddenPrints
from ml_features.features import create_features
from ml_training.train_xgb import train_xgb
from ml_simulation__discount.sample import sample_discount_customers
from ml_simulation__discount.data import get_discount_compute_function
from ml_simulation__discount.widget import show_discount_widget
    
import warnings
warnings.filterwarnings('ignore')

df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])

In [3]:
split_result = customer_split(df_quotes)
df_train = split_result['train']
df_sim = split_result['simulation']


SPLIT CUSTOMERS: TRAIN vs SIMULATION: TRAINING SIZE 0.95
Split: 22708 train, 1180 sim customers


In [4]:
TRAIN = False

In [5]:
# Model building
if TRAIN:
    with HiddenPrints():
        X_train = create_features(df_train)
    
    y_train = X_train['converted']
    X_train = X_train.drop(['numero_compte', 'converted'], axis=1)
    feature_names = X_train.columns.tolist()
    
    result = train_xgb(X_train, y_train, "simulation_poc")
    model = result['model']
    feature_names = result['features']
    
    print(f"Model trained: {len(feature_names)} features")
else:
    with open('simulation_poc.pkl', 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        feature_names = model_data['features']

In [6]:
# Sampling
sample_seed = int(time.time() * 1000) % 10000000
selected_ids = sample_discount_customers(df_sim, random_state=sample_seed)

Non-converted customers: 923
Candidates with price data: 923

📊 SAMPLING STRATEGY:
   1. No existing discount (test introduction)
   2. Small existing discount (< 2%)
   3. Medium existing discount (2-5%)
   4. High price (>€20k)
   5. Multiple quotes (≥3)
✓ Sampled no-discount: CL00345673
✓ Sampled small-discount: CL00345841
✓ Sampled medium-discount: CL00268279
✓ Sampled high-price: CL00345324
✓ Sampled multi-quote: CL00345567

🎯 SELECTED 5 DISCOUNT CUSTOMERS:
customer_id  quote_count  total_price  discount_pct
 CL00345673            2      9492.67      0.000000
 CL00345841            1     14319.49      0.698349
 CL00268279            1      6600.64      3.030009
 CL00345324            1     20370.83      0.000000
 CL00345567            3     22259.00      0.000000

Selected IDs: ['CL00345673', 'CL00345841', 'CL00268279', 'CL00345324', 'CL00345567']


In [7]:
# Simulation
compute = get_discount_compute_function(model, feature_names, df_sim, selected_ids)
show_discount_widget(compute, selected_ids)